In [ ]:
import folium
import pandas as pd
from folium.plugins import TimestampedGeoJson

# CSV laden
df = pd.read_csv("ortsnamen_georeferenziert_datiert_1613_muenchen.csv")

# Nur Zeilen mit Datum und Koordinaten
df = df.dropna(subset=['date', 'latitude', 'longitude'])

# Datum als Zeichenkette behandeln und 400 Jahre "simuliert" hinzufügen
def shift_year(date_str):
    try:
        year, month, day = map(int, date_str.split("-"))
        return f"{year + 400:04d}-{month:02d}-{day:02d}"
    except:
        return None

df['date_shifted'] = df['date'].apply(shift_year)
df = df.dropna(subset=['date_shifted'])

# GeoJSON-Features bauen
geojson_features = []
for _, row in df.iterrows():
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [float(row["longitude"]), float(row["latitude"])]
        },
        "properties": {
            "times": [row["date_shifted"]],
            "popup": f"{row.get('place_normalized', '')}<br>Originaldatum: {row['date']}",
            "tooltip": row.get("place_normalized", "")
        }
    }
    geojson_features.append(feature)

# Karte initialisieren
m = folium.Map(location=[51.0, 10.0], zoom_start=6)

# Zeitlayer hinzufügen
TimestampedGeoJson(
    {
        "type": "FeatureCollection",
        "features": geojson_features
    },
    period="P1M",
    duration="P3M",
    auto_play=False,
    loop=True,
    max_speed=1,
    loop_button=True,
    date_options='YYYY-MM-DD',
    time_slider_drag_update=True
).add_to(m)

# Karte speichern
m.save("zeitkarte_1613_muenchen.html")
print("Karte gespeichert: zeitkarte_1613_muenchen.html")